In [1]:
from sympy import *
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum import Commutator
from sympy.physics.quantum import Operator
from itertools import permutations
from itertools import combinations

In [2]:
w_0,w_1,w_2,g,t = symbols('w_0,w_1,w_2,g,t')
ft = Function('f')(t)
kt = Function('k')(t)
w_d = symbols('w_d')
 
sigmap01 = Matrix([[0,0,0],[1,0,0],[0,0,0]])
sigmap12 = Matrix([[0,0,0],[0,0,0],[0,1,0]])
sigmam01 = Matrix([[0,1,0],[0,0,0],[0,0,0]])
sigmam12 = Matrix([[0,0,0],[0,0,1],[0,0,0]])
sigmap02 = Matrix([[0,0,0],[0,0,0],[1,0,0]])
sigmam02 = Matrix([[0,0,1],[0,0,0],[0,0,0]])


H0 = diag(w_0,w_1,w_2)
H1 = 1*ft*(sigmap01 + sigmam01) 

H2 = g*ft*(sigmap12+sigmam12) #1*kt*(sigmap01 + sigmam01) + g*kt*(sigmap12+sigmam12)

# H1 = diag(0,0,0)
# H2 = 1*ft*(sigmap01 + sigmam01) + g*ft*(sigmap12+sigmam12)+ 1*kt*(sigmap01 + sigmam01) + g*kt*(sigmap12+sigmam12)

In [3]:
subq_eigen = [Matrix([[1],[0],[0]]),Matrix([[0],[1],[0]])]
subrest_eigen = [Matrix([[0],[0],[1]])]

# subq_eigen = [Matrix([[1],[0],[0]]),Matrix([[0],[0],[1]])]
# subrest_eigen = [Matrix([[0],[1],[0]])]


Pq = diag([0]*3,unpack=True)
Prest = diag([0]*3,unpack=True)
for k in [i*i.transpose() for i in subq_eigen]:
    Pq += k
for k in [i*i.transpose() for i in subrest_eigen]:
    Prest += k


In [4]:
S1_try = diag([0]*3,unpack=True)
for subpairs in permutations([subq_eigen,subrest_eigen],2):
    for i in subpairs[0]:
        for j in subpairs[1]:
            S1_try += -(i.transpose()*H2*j)[0]/(i.transpose()*H0*i-j.transpose()*H0*j)[0]*i*j.transpose()

In [5]:
# S2_try = diag([0]*3,unpack=True)
# for subpairs in permutations([subq_eigen,subrest_eigen],2):
#     for i in subpairs[0]:
#         for j in subpairs[1]:
#             S2_try += -((i.transpose()*(H1*S1_try-S1_try*H1)*j)[0]+0.5*(i.transpose()*(H2*S1_try-S1_try*H2)*j)[0])/(i.transpose()*H0*i-j.transpose()*H0*j)[0]*i*j.transpose()
S2_try = diag([0]*3,unpack=True)
for subpairs in permutations([subq_eigen,subrest_eigen],2):
    for i in subpairs[0]:
        for j in subpairs[1]:
            S2_try += -((i.transpose()*(H1*S1_try-S1_try*H1)*j)[0]+0.5*(i.transpose()*(H2*S1_try-S1_try*H2)*j)[0]-1j*(i.transpose()*diff(S1_try,t)*j)[0])/(i.transpose()*H0*i-j.transpose()*H0*j)[0]*i*j.transpose()

In [6]:
res_Scut1 = H0 + H1 + (H2*S1_try-S1_try*H2)/2
res_Scut2 = H0 + H1 + (H2*S1_try-S1_try*H2)/2 + (H2*S2_try-S2_try*H2)/2


In [7]:
# (-1j*res_Scut1).exp()
(res_Scut1)
# (res_Scut1)-1j*diff(S1_try,t)


Matrix([
[ w_0,                                                               f(t),                                                                   0],
[f(t), g**2*f(t)**2/(2*(w_1 - w_2)) - g**2*f(t)**2/(2*(-w_1 + w_2)) + w_1,                                                                   0],
[   0,                                                                  0, -g**2*f(t)**2/(2*(w_1 - w_2)) + g**2*f(t)**2/(2*(-w_1 + w_2)) + w_2]])

### 进入旋转参考系

In [8]:
# H = H0 + H1 +  H2 + 1*kt*(sigmap01 + sigmam01) + g*kt*(sigmap12+sigmam12)
H = H0 + H1 +  H2 + 1*kt*diag(0,1,2)
Hz = (w_d)*diag(0,1,2)
Hr = (1j*t*Hz).exp()*(H)*(-1j*t*Hz).exp() + 1j*diff((1j*t*Hz).exp(),t)*(-1j*t*Hz).exp()
# Hr1 = (1j*t*Hz).exp()*(H)*(-1j*t*Hz).exp()- Hz

Hr 


Matrix([
[              1.0*w_0,       1.0*f(t)*exp(-I*t*w_d),                            0],
[1.0*f(t)*exp(I*t*w_d), 1.0*w_1 - 1.0*w_d + 1.0*k(t), 1.0*g*f(t)*exp(-1.0*I*t*w_d)],
[                    0,  1.0*g*f(t)*exp(1.0*I*t*w_d), 1.0*w_2 - 2.0*w_d + 2.0*k(t)]])

### 进入去除x旋转的旋转参考系

旋转波近似下重写哈密顿量，并且重新约定符号：
\begin{equation}
f(t) := f(t)\cos(\omega_d t)
\end{equation}

\begin{equation}
H = \left(
    \begin{matrix}
    \omega_0   &   \frac{1}{2}f(t)    & 0 \\ 
    \frac{1}{2}f(t)  & \omega_1-\omega_d+k(t)  & \frac{g}{2}f(t)  \\
    0    &  \frac{g}{2}f(t) &  \omega_2-2\omega_d+2k(t)  \\
    \end{matrix}
    \right)
\end{equation}

即此时的$f(t),k(t)$为波形包络

In [9]:
# Rewrite H
Hr_rewrite = Hr.copy()
Hr_rewrite[0,1] = (ft)/2
Hr_rewrite[1,0] = (ft)/2
Hr_rewrite[1,2] = g*(ft)/2
Hr_rewrite[2,1] = g*(ft)/2
Hr_rewrite


Matrix([
[1.0*w_0,                       f(t)/2,                            0],
[ f(t)/2, 1.0*w_1 - 1.0*w_d + 1.0*k(t),                     g*f(t)/2],
[      0,                     g*f(t)/2, 1.0*w_2 - 2.0*w_d + 2.0*k(t)]])

In [10]:
# H_2 是需要消除掉的项，由于H2的选择，对角化只能是0,2空间直和1空间，而非直观上的01空间直和2空间

H0_r = diag(w_0,w_1-w_d+kt,w_2-2*w_d+2*kt)
H1_r = diag(0,0,0)
H2_r = 1*ft/2*(sigmap01 + sigmam01) + g*ft/2*(sigmap12 + sigmam12)


In [11]:
H2_r 

Matrix([
[     0,   f(t)/2,        0],
[f(t)/2,        0, g*f(t)/2],
[     0, g*f(t)/2,        0]])

In [12]:
# subq_eigen = [Matrix([[1],[0],[0]]),Matrix([[0],[1],[0]])]
# subrest_eigen = [Matrix([[0],[0],[1]])]

subq_eigen = [Matrix([[1],[0],[0]]),Matrix([[0],[0],[1]])]
subrest_eigen = [Matrix([[0],[1],[0]])]


Pq = diag([0]*3,unpack=True)
Prest = diag([0]*3,unpack=True)
for k in [i*i.transpose() for i in subq_eigen]:
    Pq += k
for k in [i*i.transpose() for i in subrest_eigen]:
    Prest += k

S1_try = diag([0]*3,unpack=True)
for subpairs in permutations([subq_eigen,subrest_eigen],2):
    for i in subpairs[0]:
        for j in subpairs[1]:
            S1_try += -(i.transpose()*H2_r*j)[0]/(i.transpose()*H0_r*i-j.transpose()*H0_r*j)[0]*i*j.transpose()

S2_try = diag([0]*3,unpack=True)
for subpairs in permutations([subq_eigen,subrest_eigen],2):
    for i in subpairs[0]:
        for j in subpairs[1]:
            S2_try += -((i.transpose()*(H1_r*S1_try-S1_try*H1_r)*j)[0]+0.5*(i.transpose()*(H2_r*S1_try-S1_try*H2_r)*j)[0]-1j*(i.transpose()*diff(S1_try,t)*j)[0])/(i.transpose()*H0_r*i-j.transpose()*H0_r*j)[0]*i*j.transpose()

In [13]:
S1_try

Matrix([
[                                  0,    -f(t)/(2*(w_0 - w_1 + w_d - k(t))),                                    0],
[-f(t)/(2*(-w_0 + w_1 - w_d + k(t))),                                     0, -g*f(t)/(2*(w_1 - w_2 + w_d - k(t)))],
[                                  0, -g*f(t)/(2*(-w_1 + w_2 - w_d + k(t))),                                    0]])

In [14]:
res_rScut1 = H0_r + H1_r + (H2_r*S1_try-S1_try*H2_r)/2 - 1j*diff(S1_try,t)

In [15]:
res_rScut1 
# k(t)没有虚部，所以在x轴方向补偿是无法抑制泄露态的
# k(t)只能写成k(t) * sin

Matrix([
[                                     w_0 + f(t)**2/(8*(w_0 - w_1 + w_d - k(t))) - f(t)**2/(8*(-w_0 + w_1 - w_d + k(t))),                                                                    -1.0*I*(-Derivative(f(t), t)/(2*(w_0 - w_1 + w_d - k(t))) - f(t)*Derivative(k(t), t)/(2*(w_0 - w_1 + w_d - k(t))**2)),                                          -g*f(t)**2/(8*(w_1 - w_2 + w_d - k(t))) + g*f(t)**2/(8*(w_0 - w_1 + w_d - k(t)))],
[-1.0*I*(-Derivative(f(t), t)/(2*(-w_0 + w_1 - w_d + k(t))) + f(t)*Derivative(k(t), t)/(2*(-w_0 + w_1 - w_d + k(t))**2)), g**2*f(t)**2/(8*(w_1 - w_2 + w_d - k(t))) - g**2*f(t)**2/(8*(-w_1 + w_2 - w_d + k(t))) + w_1 - w_d + k(t) - f(t)**2/(8*(w_0 - w_1 + w_d - k(t))) + f(t)**2/(8*(-w_0 + w_1 - w_d + k(t))), -1.0*I*(-g*Derivative(f(t), t)/(2*(w_1 - w_2 + w_d - k(t))) - g*f(t)*Derivative(k(t), t)/(2*(w_1 - w_2 + w_d - k(t))**2))],
[                                      g*f(t)**2/(8*(-w_1 + w_2 - w_d + k(t))) - g*f(t)**2/(8*(-w_0 + w_1 - w_d + k(t))),      

In [16]:
res_rScut1[2,1]

-1.0*I*(-g*Derivative(f(t), t)/(2*(-w_1 + w_2 - w_d + k(t))) + g*f(t)*Derivative(k(t), t)/(2*(-w_1 + w_2 - w_d + k(t))**2))

In [17]:
res_rScut1[1,1]-res_rScut1[0,0]

g**2*f(t)**2/(8*(w_1 - w_2 + w_d - k(t))) - g**2*f(t)**2/(8*(-w_1 + w_2 - w_d + k(t))) - w_0 + w_1 - w_d + k(t) - f(t)**2/(4*(w_0 - w_1 + w_d - k(t))) + f(t)**2/(4*(-w_0 + w_1 - w_d + k(t)))

In [18]:
res_rScut1[2,2]-res_rScut1[1,1]


-g**2*f(t)**2/(4*(w_1 - w_2 + w_d - k(t))) + g**2*f(t)**2/(4*(-w_1 + w_2 - w_d + k(t))) - w_1 + w_2 - w_d + k(t) + f(t)**2/(8*(w_0 - w_1 + w_d - k(t))) - f(t)**2/(8*(-w_0 + w_1 - w_d + k(t)))